## Demo3
In this notebook, I try to address output structure of this algorithm. The result should be a combination of degree and school, or position and company.
E.g, `xxx本科是中山大学，在清华大学获得硕士学位`. Then I expect the result to be something like: `<中山大学, 本科>`,`<清华大学, 硕士>`

Idea:
+ Link entity seperately, do linking each n-gram, thus can make use of the order of text.
+ Eliminate duplicate entities after linking

In [31]:
import jieba
import re
import chardet
from gensim.models import KeyedVectors
import numpy as np
import pandas as pd
import math
import datetime

## Load File

In [2]:
def loadSchool():
    path1 = 'data/chinese_university_list.csv'
    school_df = pd.read_csv(path1, header=None, delimiter=",", skiprows=4, names=["rank", "name", "code", "department", "city", "level", "notes"])
    school_df = pd.DataFrame(school_df, columns=['name'])
    print(school_df.shape[0])
    
    path2 = 'data/all_university.csv'
    school_global_df = pd.read_csv(path2, header=None, delimiter=',', skiprows=4, names=['Name_en', 'Name_ch', 'rank', 'score', 'location'])
    school_global_df = pd.DataFrame(school_global_df, columns=['Name_ch'])
    print(school_global_df.shape[0])
    
    school_df = pd.concat([school_df, school_global_df], axis=0, ignore_index=True)
    print(school_df.shape[0])
    return school_df

In [3]:
def loadDegree():
    degree = {'name': ['学士', '硕士', '博士']}
    degree_df = pd.DataFrame(degree)
    print(degree_df)
    return degree_df

In [28]:
def loadMember():
    path = 'data/member-data.csv'
    member_df = pd.read_csv(path, header=None, delimiter=",", skiprows=1, names=['Company', 'No.', 'Resume', 'Position'])
    member_df = pd.DataFrame(member_df, columns=['Resume'])
    print(member_df.head())
    return member_df

## Preprocess Text

In [4]:
def removeStopWords(seglist):
    stopwords = {}
    fstop = open('data/stopwords_cn.txt', 'r', encoding='utf-8', errors='ignore')
    for w in fstop:
        stopwords[w.strip()] = w.strip()

    fstop.close()
    stopwords[' '] = ' '
    
    segListSanitized = []

    for word in seglist:
        if word not in stopwords:
            segListSanitized.append(word)
    return segListSanitized

In [5]:
def preprocess(text):
    # remove punctuations
    text = re.sub(r"[\s+\.\!\/_,$%^*()?;；:【】+\"\']+|[+——！，;:。？、~@#￥%……&*（）]+", " ", text)
    text = text.lower()
    # seperate words
    words = jieba.cut(text, cut_all=False)
    seglist = list(words)
    # remove stopwords
    segListSanitized = removeStopWords(seglist)
    print(f'Before sanitize, len: {len(seglist)}. After sanitize, len: {len(segListSanitized)}')

    return segListSanitized

## N-gram Algorithm

In [6]:
def getNgrams(wordList, n):
    '''
    This function only generete N-Grams
    '''
    output = []
    for i in range (len(wordList) - n + 1):
        n_gram_temp = "".join(wordList[i:i+n])
        output.append(n_gram_temp)
    return output

In [7]:
def generateNgrams(wordList, n):
    '''
    This function genereates [1, N]-Grams
    '''
    result = set()
    for i in range(n):
        temp = getNgrams(wordList, i+1)
        result = result | temp
    
    return result

In [23]:
def generateNgramsV2(wordList, n):
    '''
    This function genereates [1, N]-Grams
    '''
    result = []
    for i in range(n):
        temp_list = getNgrams(wordList, i+1)
        temp = list(set(temp_list))
        temp.sort(key=temp_list.index)
        result.append(temp)
        
    return result

## Word Embedding

In [9]:
model = KeyedVectors.load('./test_50.bin')

In [10]:
def calculate_cosine_similarity(a, b):
    vector_a = np.mat(a)
    vector_b = np.mat(b)
    num = float(vector_a * vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num / denom
    sim = 0.5 + 0.5 * cos
    return sim

In [11]:
def generateEmbeddings(name):
    words = jieba.cut(name, cut_all=False)
    word_list = list(words)
    v = np.zeros((200))
    for word in word_list:
        if word in model.vocab:
            v += model[word]
    
    v /= len(v)
    return v

In [12]:
def preprocess_entity_list(df, model):
    '''
    df: dafaframe
    model: word embedding model
    '''
    
    df['embeddings'] = ''
    for index, row in df.iterrows():
        # df.loc[index, 'embeddings'] = z
        name = row['name']
        if isinstance(name, float):
            continue
        name = name.lower()
        if name in model.vocab:
            vec = model[name]
        else:
            vec = generateEmbeddings(name)
        df.set_value(index, 'embeddings', vec)

    # print(df.head())
    return df

## Entity Link

In [13]:
def linkSchoolAndDegree(output, model, school_df, degree_df, school_threshold, degree_threshold):
    for index, li in enumerate(output):
        print(f'process {index}-Gram')
        for term in li:
            if len(term) <= 1:
                continue
            if term in model.vocab:
                term_vec = model[term]
                school_candidate = dict()

                # Link School
                for index, row in school_df.iterrows():
                    name = row['name']
                    if isinstance(name, float):
                        continue
                    name_vec = row['embeddings']
                    sim = calculate_cosine_similarity(term_vec, name_vec)
                    if (sim > school_threshold):
                        school_candidate[name] = sim
                if len(school_candidate) != 0:
                    school_candidate = sorted(school_candidate.items(), key=lambda item:item[1], reverse=True)
                    print(f'university entity found: {term}->{school_candidate[0][0]}, sim = {school_candidate[0][1]}')

                # Link Degree
                degree_candidate = dict()
                for index, row in degree_df.iterrows():
                    name = row['name']
                    name_vec = row['embeddings']
                    sim = calculate_cosine_similarity(term_vec, name_vec)
                    if (sim > degree_threshold):
                        degree_candidate[name] = sim
                if len(degree_candidate) != 0:
                    degree_candidate = sorted(degree_candidate.items(), key=lambda item:item[1], reverse=True)
                    print(f'degree entity found: {term}->{degree_candidate[0][0]}, sim = {degree_candidate[0][1]}')

## Main Function

In [35]:
school_df = loadSchool()
degree_df = loadDegree()
member_df = loadMember()

2718
949
3667
  name
0   学士
1   硕士
2   博士
                                              Resume
0  __团队成员#1__先生是公司创始人,也是中国最有影响力的商界领袖之一。1982年,__团队...
1  __团队成员#2__先生,现任TCL集团股份有限公司执行董事、总裁(COO)。1963年4月...
2  __团队成员#3__女士:1972年7月出生,中山大学法学博士,高级经济师。1993年6月至...
3  __团队成员#4__先生,1965年7月出生,东方电气集团党组副书记、副总经理,兼任东方电气...
4  __团队成员#5__女士,现任TCL多媒体集团有限公司非执行独立董事、A8新媒体集团非执行独...


c:\users\梁育诚\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [36]:
school_df = preprocess_entity_list(school_df, model)
degree_df = preprocess_entity_list(degree_df, model)

c:\users\梁育诚\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [16]:
text = '__团队成员#1__先生是公司创始人,也是中国最有影响力的商界领袖之一。1982年,__团队成员#1__先生于华南理工大学毕业,进入TCL的前身-TTK家庭电器有限公司。1985年,他担任新成立的TCL通讯设备公司总经理,创立了TCL品牌。2003年,__团队成员#1__担任TCL集团股份有限公司董事长兼CEO,随后TCL集团整体上市。在他的领导下,2004年TCL一举收购了法国汤姆逊全球彩电业务与阿尔卡特全球手机业务。目前TCL集团已经成为拥有6万名员工,业务遍及全球80多个国家和地区。2013年,TCL集团营业总收入超过855亿元,液晶电视全球销量1766万台,实际产量全球第三,品牌销售全球第三;TCL手机全球销量5520万台,行业排名全球第五。2012年__团队成员#1__被新华网评为“最具社会责任感企业家”;2011年荣获《中国企业家》“最具影响力的25位企业领袖”终身成就奖;2009年被评为“CCTV中国经济年度人物十年商业领袖”;2008年获改革开放30年经济人物称号;2004年被评为Fortune杂志“亚洲年度经济人物”、TIME杂志和CNN全球最具影响力的25名商界人士,同年法国总统希拉克向__团队成员#1__先生颁发了法国国家荣誉勋章。__团队成员#1__是中共第十六大代表,第十届、第十一届、第十二届全国人大代表。__团队成员#1__担任的社会职务包括:中国电子视像行业协会会长;中国国际商会副会长;全国工商联执行委员、广东省工商联(总商会)副主席。'

In [17]:
segListSanitized = preprocess(text)
output = generateNgramsV2(segListSanitized, 3)
linkSchoolAndDegree(output, model, school_df, degree_df, 0.9, 0.98)
print()

Before sanitize, len: 325. After sanitize, len: 239
process 0-Gram
university entity found: 华南理工大学->华南理工大学, sim = 1.000000029802326
process 1-Gram
process 2-Gram



In [18]:
text2 = '__团队成员#2__先生,现任TCL集团股份有限公司执行董事、总裁(COO)。1963年4月出生,博士,毕业于西安交通大学。薄先生1988年至1993年间任陕西财经学院贸易经济系副主任,1993年5月至2000年5月间任深圳航空公司总会计师,是深圳航空公司创始人之一。薄先生2000年5月至2004年4月间任TCL信息产业集团副总裁、财务总监,2004年4月至2005年1月间任TCL集团部品事业本部副总裁,2005年1月至2005年10月间任TCL集团股份有限公司人力资源部部长、总裁办主任,2005年10月至2006年11月间任TCL集团股份有限公司控股子公司TTE Corporation执行副总裁,2006年6月至2007年9月间任TCL集团股份有限公司人力资源总监,2006年8月至2007年10月间任TCL集团股份有限公司副总裁,2007年10月至2008年6月间任高级副总裁,2008年6月至2011年6月,任TCL集团股份有限公司首席运营官,2011年6月至今任TCL集团股份有限公司总裁(COO),2012年12月24日起任深圳市华星光电技术有限公司CEO。__团队成员#2__先生现兼任TCL多媒体科技控股有限公司执行董事、深圳市华星光电技术有限公司董事、翰林汇信息产业股份有限公司董事长、TCL教育网有限公司董事长、电大在线远程教育技术有限公司副董事长。__团队成员#2__先生于二零一四年十月二十三日调任为TCL多媒体科技控股有限公司之执行董事，并获委任为策略执行委员会主席。__团队成员#2__先生获委任为TCL多媒体科技控股有限公司首席执行官,自二零一五年六月二十五日起生效。'

In [19]:
segListSanitized = preprocess(text2)
output = generateNgramsV2(segListSanitized, 3)
linkSchoolAndDegree(output, model, school_df, degree_df, 0.9, 0.90)
print()

Before sanitize, len: 330. After sanitize, len: 264
process 0-Gram
degree entity found: 博士->博士, sim = 1.0
university entity found: 西安交通大学->西安交通大学, sim = 1.0000000596046448
university entity found: 财经学院->宁波财经学院, sim = 0.9081512997142207
process 1-Gram
degree entity found: 博士毕业->硕士, sim = 0.9167363941669464
process 2-Gram



In [24]:
text3 = '__团队成员#6__先生,1980年7月生,硕士研究生学历。2002年,福州大学经济学本科毕业;2006年,云南大学法律硕士研究生毕业。2006年8月至2014年2月,任职国泰君安证券股份有限公司,历任国泰君安证券香港公司财务顾问部高级经理、总经理,深圳总部机构客户部总监,从事香港与中国资本市场的投资银行业务。2014年3月加入TCL集团股份有限公司,任公司董事会办公室主任;2014年4月起任公司董事会秘书;2014年12月起任公司执委会成员;2015年4月起任TCL集团控股子公司全球播有限公司董事;2015年5月起任TCL通讯科技控股有限公司(02618.HK)非执行董事。'

In [25]:
segListSanitized = preprocess(text3)
output = generateNgramsV2(segListSanitized, 3)
linkSchoolAndDegree(output, model, school_df, degree_df, 0.9, 0.90)
print()

Before sanitize, len: 139. After sanitize, len: 111
process 0-Gram
degree entity found: 硕士->硕士, sim = 1.0
degree entity found: 研究生->硕士, sim = 0.9259832799434662
university entity found: 福州大学->福州大学, sim = 1.0000000596046448
degree entity found: 本科毕业->硕士, sim = 0.9262227118015289
university entity found: 云南大学->云南大学, sim = 1.0000000596046448
process 1-Gram
degree entity found: 硕士研究生->硕士, sim = 0.9260654151439667
degree entity found: 研究生毕业->硕士, sim = 0.9024443626403809
process 2-Gram



In [22]:
print(output[0])

['团队', '成员', '6', '先生', '1980', '年', '7', '月生', '硕士', '研究生', '学历', '2002', '年', '福州大学', '经济学', '本科毕业', '2006', '年', '云南大学', '法律硕士', '研究生', '毕业', '2006', '年', '8', '月', '2014', '年', '2', '月', '任职', '国泰君安证券股份有限公司', '历任', '国泰君安', '证券', '香港', '公司', '财务顾问', '部', '高级', '经理', '总经理', '深圳', '总部', '机构', '客户部', '总监', '从事', '香港', '中国', '资本', '市场', '投资', '银行业务', '2014', '年', '3', '月', '加入', 'tcl', '集团股份', '有限公司', '任', '公司', '董事会', '办公室', '主任', '2014', '年', '4', '月', '起任', '公司', '董事会', '秘书', '2014', '年', '12', '月', '起任', '公司', '执委会', '成员', '2015', '年', '4', '月', '起任', 'tcl', '集团', '控股', '子公司', '全球', '播', '有限公司', '董事', '2015', '年', '5', '月', '起任', 'tcl', '通讯', '科技', '控股', '有限公司', '02618', 'hk', '非', '执行', '董事']


In [37]:
for index, row in member_df.iterrows():
    if index == 40:
        break
    print(f'Handle No.{index} text')
    start = datetime.datetime.now()
    text = row['Resume']
    segListSanitized = preprocess(text)
    output = generateNgramsV2(segListSanitized, 3)
    linkSchoolAndDegree(output, model, school_df, degree_df, 0.9, 0.91)
    end = datetime.datetime.now()
    print(f'cost time: {end - start} sec')
    print()
    

Handle No.0 text
Before sanitize, len: 325. After sanitize, len: 239
process 0-Gram
university entity found: 华南理工大学->华南理工大学, sim = 1.000000029802326
process 1-Gram
process 2-Gram
cost time: 0:03:43.105396 sec

Handle No.1 text
Before sanitize, len: 330. After sanitize, len: 264
process 0-Gram
degree entity found: 博士->博士, sim = 1.0
university entity found: 西安交通大学->西安交通大学, sim = 1.0000000596046448
university entity found: 财经学院->宁波财经学院, sim = 0.9081512997142207
process 1-Gram
degree entity found: 博士毕业->硕士, sim = 0.9167363941669464
process 2-Gram
cost time: 0:02:46.263789 sec

Handle No.2 text
Before sanitize, len: 134. After sanitize, len: 108
process 0-Gram
university entity found: 中山大学->中山大学, sim = 1.0
process 1-Gram
process 2-Gram
cost time: 0:01:09.030211 sec

Handle No.3 text
Before sanitize, len: 187. After sanitize, len: 149
process 0-Gram
university entity found: 大学->吉林外国语大学, sim = 0.9060334598685178
degree entity found: 本科毕业->硕士, sim = 0.9262227118015289
university entity found: 

degree entity found: 本科毕业->硕士, sim = 0.9262227118015289
university entity found: 山东大学->山东大学, sim = 1.0000000596046448
process 1-Gram
process 2-Gram
cost time: 0:01:48.303338 sec

Handle No.30 text
Before sanitize, len: 119. After sanitize, len: 89
process 0-Gram
degree entity found: 硕士->硕士, sim = 1.0
university entity found: 大学->吉林外国语大学, sim = 0.9060334598685178
process 1-Gram
degree entity found: 硕士学历->硕士, sim = 0.9123177826404572
process 2-Gram
cost time: 0:00:58.979360 sec

Handle No.31 text
Before sanitize, len: 99. After sanitize, len: 74
process 0-Gram
university entity found: 暨南大学->暨南大学, sim = 1.0
degree entity found: 教授->博士, sim = 0.9146097251540259
university entity found: 中国人民大学->中国人民大学, sim = 1.0
university entity found: 商学院->郑州商学院, sim = 0.9071498487162017
degree entity found: 博士->博士, sim = 1.0
university entity found: 大学->吉林外国语大学, sim = 0.9060334598685178
process 1-Gram
process 2-Gram
cost time: 0:01:11.983107 sec

Handle No.32 text
Before sanitize, len: 12. After sanitize